In [1]:
import pandas as pd
import numpy as np

In [2]:
census_df = pd.read_csv('co-est2019-alldata.csv', encoding='latin-1')
census_df = census_df[census_df.SUMLEV == 50]
census_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57376,...,1.020788,0.208812,-1.650165,-0.347225,-2.049590,-1.338525,-1.391062,6.193562,-0.069229,1.124597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43574,...,-17.605427,0.502513,15.035735,-4.614071,-13.140225,-13.762031,-18.484164,-23.515727,-18.743773,-17.065527
3189,50,4,8,56,39,Wyoming,Teton County,21294,21298,21296,...,-1.583464,-2.575509,1.905293,23.259519,12.508872,4.670450,1.253214,-0.171608,-12.432212,1.797445
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21121,21089,...,-9.181105,-18.196975,-4.487730,-10.870861,-15.033634,-10.484550,-11.485101,-18.923455,-13.554993,-9.181105
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8528,8530,...,-7.652085,-12.839390,-3.084589,-1.307811,-18.938032,0.000000,-15.204038,-15.950541,-17.246806,-8.289759


In [3]:
combine_fips = lambda a, b: '{:0>2}{:0>3}'.format(a, b)
census_df['FIPS'] = census_df.STATE.combine(census_df.COUNTY, combine_fips).astype(int)
census_df = census_df[['FIPS', 'POPESTIMATE2019']].set_index('FIPS')

fip_pop_dict = census_df.POPESTIMATE2019.to_dict()
census_df

,POPESTIMATE2019
FIPS,
1001,55869
1003,223234
1005,24686
1007,22394
1009,57826
...,...
56037,42343
56039,23464
56041,20226


In [4]:
# JHU Dataset 
# https://github.com/CSSEGISandData/COVID-19
# set dates based files in COVID-19\csse_covid_19_data\csse_covid_19_daily_reports_us
dates = [f'04-{12+i:02d}-2020' for i in range(19)]
dates += [f'05-{1+i:02d}-2020' for i in range(7)]

def get_jhu_table(date):
    df = pd.read_csv(f'git/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/{date}.csv')
    df['date'] = pd.to_datetime(date)
    
    df.dropna(subset=['FIPS'], inplace=True)
    df.FIPS = df.FIPS.astype(int)
    df = df[df.FIPS.isin(census_df.index)]
    
    return df
    
jhu_corona_df = pd.concat([get_jhu_table(i) for i in dates])
jhu_corona_df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,date
0,45001,Abbeville,South Carolina,US,2020-04-12 23:18:00,34.223334,-82.461707,9,0,0,9,"Abbeville, South Carolina, US",2020-04-12
1,22001,Acadia,Louisiana,US,2020-04-12 23:18:00,30.295065,-92.414197,99,5,0,94,"Acadia, Louisiana, US",2020-04-12
2,51001,Accomack,Virginia,US,2020-04-12 23:18:00,37.767072,-75.632346,15,0,0,15,"Accomack, Virginia, US",2020-04-12
3,16001,Ada,Idaho,US,2020-04-12 23:18:00,43.452658,-116.241552,517,6,0,511,"Ada, Idaho, US",2020-04-12
4,19001,Adair,Iowa,US,2020-04-12 23:18:00,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US",2020-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,4027,Yuma,Arizona,US,2020-05-08 02:32:32,32.768957,-113.906667,148,1,0,147,"Yuma, Arizona, US",2020-05-07
2947,8125,Yuma,Colorado,US,2020-05-08 02:32:32,40.003468,-102.425867,10,0,0,10,"Yuma, Colorado, US",2020-05-07
2948,48505,Zapata,Texas,US,2020-05-08 02:32:32,27.001564,-99.169872,7,0,0,7,"Zapata, Texas, US",2020-05-07
2949,48507,Zavala,Texas,US,2020-05-08 02:32:32,28.866172,-99.760508,1,0,0,1,"Zavala, Texas, US",2020-05-07


In [5]:
jhu_pivot = jhu_corona_df.pivot('FIPS', 'date').Confirmed
jhu_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,24.0,26.0,26.0,25.0,26.0,28.0,30.0,...,40.0,43.0,44.0,42.0,45.0,48.0,53.0,53.0,58.0,61.0
1003,71.0,72.0,87.0,91.0,101.0,103.0,109.0,112.0,117.0,123.0,...,171.0,174.0,174.0,175.0,181.0,187.0,188.0,189.0,196.0,205.0
1005,10.0,10.0,11.0,12.0,14.0,15.0,18.0,20.0,22.0,28.0,...,37.0,37.0,39.0,42.0,43.0,45.0,45.0,47.0,47.0,51.0
1007,16.0,17.0,17.0,18.0,22.0,24.0,26.0,28.0,32.0,32.0,...,42.0,42.0,42.0,42.0,42.0,43.0,42.0,43.0,43.0,44.0
1009,13.0,14.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,...,34.0,36.0,37.0,39.0,40.0,40.0,40.0,40.0,42.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,16.0,...,16.0,17.0,18.0,18.0,18.0,18.0,18.0,18.0,19.0,19.0
56039,56.0,56.0,57.0,58.0,59.0,61.0,62.0,62.0,62.0,92.0,...,96.0,96.0,96.0,96.0,97.0,98.0,98.0,98.0,98.0,98.0


In [6]:
# NYTimes Data
# https://github.com/nytimes/covid-19-data
start_date_nyt = '2020-04-12' # Match the John Hopkins Data

nyt_corona_df = pd.read_csv('git/covid-19-data/us-counties.csv')
nyt_corona_df.date = pd.to_datetime(nyt_corona_df.date)

nyt_corona_df.rename(columns={'fips': 'FIPS'}, inplace=True)
nyt_corona_df.dropna(subset=['FIPS'], inplace=True)
nyt_corona_df.FIPS = nyt_corona_df.FIPS.astype(int)
nyt_corona_df = nyt_corona_df[nyt_corona_df.FIPS.isin(census_df.index)]

nyt_corona_df = nyt_corona_df[nyt_corona_df.date >= start_date_nyt]
nyt_corona_df

,date,county,state,FIPS,cases,deaths
51046,2020-04-12,Autauga,Alabama,1001,19,1
51047,2020-04-12,Baldwin,Alabama,1003,71,1
51048,2020-04-12,Barbour,Alabama,1005,10,0
51049,2020-04-12,Bibb,Alabama,1007,16,0
51050,2020-04-12,Blount,Alabama,1009,13,0
...,...,...,...,...,...,...
123944,2020-05-07,Sublette,Wyoming,56035,3,0
123945,2020-05-07,Sweetwater,Wyoming,56037,19,0
123946,2020-05-07,Teton,Wyoming,56039,98,1
123947,2020-05-07,Uinta,Wyoming,56041,8,0


In [7]:
nyt_pivot = nyt_corona_df.pivot('FIPS', 'date').cases
nyt_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,25.0,25.0,26.0,25.0,27.0,28.0,30.0,...,40.0,43.0,42.0,42.0,45.0,48.0,53.0,53.0,58.0,61.0
1003,71.0,78.0,87.0,98.0,102.0,103.0,109.0,114.0,117.0,123.0,...,171.0,173.0,174.0,175.0,181.0,187.0,188.0,189.0,196.0,205.0
1005,10.0,10.0,11.0,13.0,14.0,15.0,18.0,20.0,22.0,28.0,...,37.0,37.0,39.0,42.0,43.0,45.0,45.0,47.0,47.0,51.0
1007,16.0,17.0,17.0,19.0,24.0,23.0,26.0,28.0,32.0,32.0,...,42.0,42.0,42.0,42.0,42.0,43.0,42.0,43.0,43.0,44.0
1009,13.0,15.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,...,34.0,36.0,37.0,39.0,40.0,40.0,40.0,40.0,42.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,16.0,17.0,18.0,18.0,18.0,18.0,18.0,18.0,19.0,19.0
56039,56.0,56.0,57.0,59.0,59.0,61.0,62.0,62.0,62.0,62.0,...,96.0,95.0,96.0,96.0,97.0,98.0,98.0,98.0,98.0,98.0


In [8]:
#combine John Hopkins and NYT data, prioritizing John Hopkins
take_first = lambda s1, s2: s1 if s1 is not None else s2
corona_pivot = jhu_pivot.combine(nyt_pivot, take_first)
corona_pivot = corona_pivot.bfill(axis = 1).ffill(axis = 1).dropna()
corona_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,24.0,26.0,26.0,25.0,26.0,28.0,30.0,...,40.0,43.0,44.0,42.0,45.0,48.0,53.0,53.0,58.0,61.0
1003,71.0,72.0,87.0,91.0,101.0,103.0,109.0,112.0,117.0,123.0,...,171.0,174.0,174.0,175.0,181.0,187.0,188.0,189.0,196.0,205.0
1005,10.0,10.0,11.0,12.0,14.0,15.0,18.0,20.0,22.0,28.0,...,37.0,37.0,39.0,42.0,43.0,45.0,45.0,47.0,47.0,51.0
1007,16.0,17.0,17.0,18.0,22.0,24.0,26.0,28.0,32.0,32.0,...,42.0,42.0,42.0,42.0,42.0,43.0,42.0,43.0,43.0,44.0
1009,13.0,14.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,...,34.0,36.0,37.0,39.0,40.0,40.0,40.0,40.0,42.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,16.0,...,16.0,17.0,18.0,18.0,18.0,18.0,18.0,18.0,19.0,19.0
56039,56.0,56.0,57.0,58.0,59.0,61.0,62.0,62.0,62.0,92.0,...,96.0,96.0,96.0,96.0,97.0,98.0,98.0,98.0,98.0,98.0


In [9]:
# set how many days it takes for an average case to resolve
num_days = 14

In [10]:
sir_df = pd.DataFrame(corona_pivot.iloc[:, -1].rename("cases_now"))
sir_df['cases_old'] = corona_pivot.iloc[:, -num_days]
display(sir_df.describe())
sir_df

,cases_now,cases_old
count,2896.000000,2896.000000
mean,430.077348,309.667818
std,3890.280057,3165.224849
min,0.000000,0.000000
25%,7.000000,4.000000
50%,27.000000,17.000000
75%,117.000000,75.000000
max,180216.000000,150473.000000


,cases_now,cases_old
FIPS,,
1001,61.0,36.0
1003,205.0,147.0
1005,51.0,32.0
1007,44.0,34.0
1009,44.0,31.0
...,...,...
56035,3.0,3.0
56037,19.0,16.0
56039,98.0,95.0


In [11]:
sir_df['POPULATION'] = [fip_pop_dict[i] for i in sir_df.index]
sir_df['SUSCEPTIBLE'] = (sir_df.POPULATION - sir_df.cases_now)/sir_df.POPULATION
sir_df['INFECTED'] = (sir_df.cases_now - sir_df.cases_old)/sir_df.POPULATION
sir_df['REMOVED'] = (sir_df.cases_old)/sir_df.POPULATION
sir_df.SUSCEPTIBLE = [float('NaN') if i>1 or i<0 else i for i in sir_df.SUSCEPTIBLE]
sir_df.INFECTED = [float('NaN') if i>1 or i<0 else i for i in sir_df.INFECTED]
sir_df.REMOVED = [float('NaN') if i>1 or i<0 else i for i in sir_df.REMOVED]
sir_df = sir_df[['SUSCEPTIBLE', 'INFECTED', 'REMOVED']].dropna()
sir_df

,SUSCEPTIBLE,INFECTED,REMOVED
FIPS,,,
1001,0.998908,0.000447,0.000644
1003,0.999082,0.000260,0.000659
1005,0.997934,0.000770,0.001296
1007,0.998035,0.000447,0.001518
1009,0.999239,0.000225,0.000536
...,...,...,...
56035,0.999695,0.000000,0.000305
56037,0.999551,0.000071,0.000378
56039,0.995823,0.000128,0.004049


In [12]:
# calculate number of people infected everyday
corona_pivot_diff = corona_pivot.diff(axis=1)
corona_pivot_diff.dropna(axis=1, how='all', inplace=True)
corona_pivot_diff

date,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,...,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0.0,4.0,1.0,2.0,0.0,-1.0,1.0,2.0,2.0,2.0,...,1.0,3.0,1.0,-2.0,3.0,3.0,5.0,0.0,5.0,3.0
1003,1.0,15.0,4.0,10.0,2.0,6.0,3.0,5.0,6.0,9.0,...,3.0,3.0,0.0,1.0,6.0,6.0,1.0,1.0,7.0,9.0
1005,0.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,6.0,1.0,...,2.0,0.0,2.0,3.0,1.0,2.0,0.0,2.0,0.0,4.0
1007,1.0,0.0,1.0,4.0,2.0,2.0,2.0,4.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,1.0
1009,1.0,2.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,3.0,...,0.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56037,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
56039,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,30.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [13]:
# normalize by dividing by population
corona_pivot_diff_pct = corona_pivot_diff.apply(lambda x: x/[fip_pop_dict[i] for i in x.index])
corona_pivot_diff_pct

date,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,...,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0.000000,0.000072,0.000018,0.000036,0.000000,-0.000018,0.000018,0.000036,0.000036,0.000036,...,0.000018,0.000054,0.000018,-0.000036,0.000054,0.000054,0.000089,0.000000,0.000089,0.000054
1003,0.000004,0.000067,0.000018,0.000045,0.000009,0.000027,0.000013,0.000022,0.000027,0.000040,...,0.000013,0.000013,0.000000,0.000004,0.000027,0.000027,0.000004,0.000004,0.000031,0.000040
1005,0.000000,0.000041,0.000041,0.000081,0.000041,0.000122,0.000081,0.000081,0.000243,0.000041,...,0.000081,0.000000,0.000081,0.000122,0.000041,0.000081,0.000000,0.000081,0.000000,0.000162
1007,0.000045,0.000000,0.000045,0.000179,0.000089,0.000089,0.000089,0.000179,0.000000,0.000089,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000045,-0.000045,0.000045,0.000000,0.000045
1009,0.000017,0.000035,0.000017,0.000017,0.000035,0.000000,0.000017,0.000017,0.000069,0.000052,...,0.000000,0.000035,0.000017,0.000035,0.000017,0.000000,0.000000,0.000000,0.000035,0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000203,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56037,0.000047,0.000000,0.000024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000142,0.000000,...,0.000000,0.000024,0.000024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000024,0.000000
56039,0.000000,0.000043,0.000043,0.000043,0.000085,0.000043,0.000000,0.000000,0.001279,0.000043,...,0.000043,0.000000,0.000000,0.000000,0.000043,0.000043,0.000000,0.000000,0.000000,0.000000


In [14]:
# Calculate average change 

def func(x):
    change_pcts = [i for i in x if i != 0]
    avg = np.sum(change_pcts)/len(x)
    
    # drop samples without too few changes or negative change
    if len(change_pcts) < 3 or avg <= 0:
        return float("NaN")
    
    return avg

average_row = lambda x: func(x)

hyperparam_df = pd.DataFrame(corona_pivot_diff_pct.iloc[:, -num_days:].T.apply(average_row).rename("beta"))
hyperparam_df['gamma'] = corona_pivot_diff_pct.iloc[:, :-num_days].T.apply(average_row)
corona_quartiles = pd.concat([sir_df.describe(), hyperparam_df.describe()], axis=1)
display(hyperparam_df)
corona_quartiles

,beta,gamma
FIPS,,
1001,0.000036,0.000023
1003,0.000020,0.000029
1005,0.000061,0.000074
1007,0.000035,0.000069
1009,0.000016,0.000028
...,...,...
56035,NaN,NaN
56037,0.000005,0.000019
56039,0.000015,0.000143


,SUSCEPTIBLE,INFECTED,REMOVED,beta,gamma
count,2869.000000,2869.000000,2869.000000,1.986000e+03,1.775000e+03
mean,0.997913,0.000787,0.001299,8.555543e-05,7.408072e-05
std,0.004949,0.002967,0.003170,2.595782e-04,1.879941e-04
min,0.879830,0.000000,0.000000,9.736586e-07,1.232048e-21
25%,0.997968,0.000066,0.000260,1.454556e-05,1.507498e-05
50%,0.999161,0.000233,0.000549,3.477415e-05,3.059919e-05
75%,0.999600,0.000716,0.001257,8.063323e-05,7.228285e-05
max,1.000000,0.117600,0.092388,8.412670e-03,4.320695e-03


In [15]:
fip_pop_pct_dict = census_df.POPESTIMATE2019.rank(pct=True).to_dict()

fip_s_dict = sir_df.SUSCEPTIBLE.to_dict()
fip_i_dict = sir_df.INFECTED.to_dict()
fip_r_dict = sir_df.REMOVED.to_dict()
fip_beta_dict = hyperparam_df.beta.dropna().to_dict()
fip_gamma_dict = hyperparam_df.gamma.dropna().to_dict()

def generate_data(fip, mode, dict1, dict2):
    try:
        return dict1[fip]
    except KeyError:
        # Generate default values based on infection rate percentile
        return dict2[fip]*(corona_quartiles[mode]['75%']-corona_quartiles[mode]['25%']) + corona_quartiles[mode]['25%']
    
matrix_df = census_df.copy()
matrix_df['SUSCEPTIBLE'] = [generate_data(i, 'SUSCEPTIBLE', fip_s_dict, fip_pop_pct_dict) for i in matrix_df.index]
matrix_df['INFECTED'] = [generate_data(i, 'INFECTED', fip_i_dict, fip_pop_pct_dict) for i in matrix_df.index]
matrix_df['REMOVED'] = [generate_data(i, 'REMOVED', fip_r_dict, fip_pop_pct_dict) for i in matrix_df.index]

inf_dict = matrix_df.INFECTED.rank(pct=True).to_dict()
rem_dict = matrix_df.REMOVED.rank(pct=True).to_dict()

matrix_df['beta'] = [generate_data(i, 'beta', fip_beta_dict, inf_dict) for i in matrix_df.index]
matrix_df['gamma'] = [generate_data(i, 'gamma', fip_gamma_dict, rem_dict) for i in matrix_df.index]

matrix_df['US'] = 1
matrix_df.drop(columns=['POPESTIMATE2019'], inplace=True)

display(matrix_df.describe())
matrix_df

,SUSCEPTIBLE,INFECTED,REMOVED,beta,gamma,US
count,3142.000000,3142.000000,3142.000000,3.142000e+03,3.142000e+03,3142.0
mean,0.997939,0.000733,0.001222,6.546725e-05,5.575349e-05,1.0
std,0.004730,0.002841,0.003040,2.081605e-04,1.430451e-04,0.0
min,0.879830,0.000000,0.000000,9.736586e-07,1.232048e-21,1.0
25%,0.998007,0.000073,0.000276,1.974086e-05,1.879386e-05,1.0
50%,0.999051,0.000205,0.000504,3.086295e-05,3.026890e-05,1.0
75%,0.999573,0.000640,0.001141,5.403410e-05,4.863586e-05,1.0
max,1.000000,0.117600,0.092388,8.412670e-03,4.320695e-03,1.0


,SUSCEPTIBLE,INFECTED,REMOVED,beta,gamma,US
FIPS,,,,,,
1001,0.998908,0.000447,0.000644,0.000036,0.000023,1
1003,0.999082,0.000260,0.000659,0.000020,0.000029,1
1005,0.997934,0.000770,0.001296,0.000061,0.000074,1
1007,0.998035,0.000447,0.001518,0.000035,0.000069,1
1009,0.999239,0.000225,0.000536,0.000016,0.000028,1
...,...,...,...,...,...,...
56037,0.999551,0.000071,0.000378,0.000005,0.000019,1
56039,0.995823,0.000128,0.004049,0.000015,0.000143,1
56041,0.999604,0.000049,0.000346,0.000028,0.000035,1


### Combine param matrix with county matrix and export

In [16]:
matrix_dict = dict(zip(matrix_df.index, matrix_df.to_numpy()))
matrix_dict

{1001: array([9.98908160e-01, 4.47475344e-04, 6.44364496e-04, 3.57980275e-05,
        2.27805630e-05, 1.00000000e+00]),
 1003: array([9.99081681e-01, 2.59817053e-04, 6.58501841e-04, 1.98382479e-05,
        2.93210467e-05, 1.00000000e+00]),
 1005: array([9.97934052e-01, 7.69667018e-04, 1.29628129e-03, 6.07631856e-05,
        7.36523462e-05, 1.00000000e+00]),
 1007: array([9.98035188e-01, 4.46548183e-04, 1.51826382e-03, 3.50859286e-05,
        6.90119918e-05, 1.00000000e+00]),
 1009: array([9.99239097e-01, 2.24812368e-04, 5.36091032e-04, 1.60580263e-05,
        2.82980603e-05, 1.00000000e+00]),
 1011: array([9.98217998e-01, 5.94000594e-04, 1.18800119e-03, 4.24286139e-05,
        7.20000720e-05, 1.00000000e+00]),
 1013: array([9.92030029e-01, 6.89016865e-03, 1.07980255e-03, 4.99500500e-04,
        5.60936390e-05, 1.00000000e+00]),
 1015: array([9.98943708e-01, 2.72875314e-04, 7.83416223e-04, 2.01198388e-05,
        2.32064050e-05, 1.00000000e+00]),
 1017: array([9.90737956e-01, 7.81860829

In [17]:
county_np = np.loadtxt('county_matrix.csv', delimiter=',', dtype=int)
county_np.shape

(1792, 2944)

In [18]:
matrix = np.zeros([county_np.shape[0], county_np.shape[1], 6])

for i in range(county_np.shape[0]):
    for j in range(county_np.shape[1]):
        try:
            # set parameter values based on the county
            matrix[i, j] = matrix_dict[county_np[i, j]]
            # Add noise to the matrix
            matrix[i, j, :5] = matrix[i, j, :5] * np.random.uniform(0.9, 1.1, 5)
        except KeyError:
            pass
        
matrix.shape

(1792, 2944, 6)

In [19]:
np.save('matrix', matrix)